In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
import ast


import os
import re
from matplotlib import pyplot as plt

In [ ]:
import string
import nltk
from collections import Counter
import matplotlib.pyplot as plt
# from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from nltk.stem import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
path = "ADMISSIONS.csv.gz"
df_adm = pd.read_csv(path, compression='gzip', header=0)
df_adm.drop_duplicates(inplace=True)
df_adm = df_adm.sort_values('ADMITTIME', ascending = True)
# df_adm.dropna(subset=['DIAGNOSIS'],inplace=True)
df_adm.dropna(subset=['HADM_ID','SUBJECT_ID'],inplace=True)
df_adm = df_adm.reset_index().drop('index',axis=1)
print(len(df_adm))
print(df_adm.HADM_ID.nunique())
df_adm.head(2)

In [ ]:
df_adm.ADMISSION_TYPE.unique()

In [ ]:
df = df_adm
# Convert datetime columns to pandas datetime format
df["ADMITTIME"] = pd.to_datetime(df["ADMITTIME"], errors='coerce')
df["DISCHTIME"] = pd.to_datetime(df["DISCHTIME"], errors='coerce')
df["DEATHTIME"] = pd.to_datetime(df["DEATHTIME"], errors='coerce')

# Extract only patients admitted to ED
# df_mortality = df[df["HOSPITAL_EXPIRE_FLAG"] == 1].reset_index()
df = df[(df.ADMISSION_TYPE.isin(['EMERGENCY'])) & (df.ADMISSION_LOCATION.isin(['EMERGENCY ROOM ADMIT']))]
# Display the extracted in-hospital mortality data
df.groupby('HOSPITAL_EXPIRE_FLAG').size()
df.groupby(['HOSPITAL_EXPIRE_FLAG','ADMISSION_TYPE','ADMISSION_LOCATION']).size()
# display(df)

## Diagnosos

In [ ]:
path = "DIAGNOSES_ICD.csv.gz"

df_diag = pd.read_csv(path, compression='gzip', header=0)
print(df_diag.ICD9_CODE.isna().sum())

df_diag.dropna(subset=['ICD9_CODE'],axis=0,inplace=True)
df_diag['len_code'] = df_diag['ICD9_CODE'].apply(lambda x: len(x))
df_diag['ICD9_CODE'] = pd.to_numeric(df_diag['ICD9_CODE'], errors='coerce')
df_diag.head(10)

In [ ]:
df = pd.merge(df,df_diag, how='left', on=['SUBJECT_ID','HADM_ID'])
print(len(df))

In [ ]:
round(df['ICD9_CODE'][df["HOSPITAL_EXPIRE_FLAG"] == 1].value_counts(normalize=True)*100,3).head(5)

In [ ]:
'''
ICD-9 Code	Description	Prevalence (%)
518.81	Acute respiratory failure	3.291
401.9	Unspecified essential hypertension	3.117
427.31	Atrial fibrillation	2.955
428.0	Congestive heart failure, unspecified	2.897
584.9	Acute kidney failure, unspecified	2.683
'''

In [ ]:
round(df['ICD9_CODE'][df["HOSPITAL_EXPIRE_FLAG"] == 0].value_counts(normalize=True)*100,3).head(5)

In [ ]:
'''
ICD-9 Code	Description	Prevalence (%)
401.9	Unspecified essential hypertension	3.678
428.0	Congestive heart failure, unspecified	2.564
427.31	Atrial fibrillation	2.149
584.9	Acute kidney failure, unspecified	1.959
414.01	Coronary atherosclerosis of native coronary artery	1.626
'''

## Focus only on Acute kidney failure, unspecified

In [ ]:
df = df[df['ICD9_CODE'] == 5849.0 ].reset_index().drop(columns=["index"])
print(len(df))

In [ ]:
df.groupby('HOSPITAL_EXPIRE_FLAG').size()

## Notes

In [ ]:
path = "NOTEEVENTS.csv.gz"
# path = "/Users/fatemehshah-mohammadi/Documents/Utah_University/Projects/MIMIC-III/NOTEEVENTS.csv.gz"

note = pd.read_csv(path, compression='gzip', header=0)
note.drop_duplicates(inplace=True)

In [ ]:
note.dropna(subset=['TEXT'],inplace=True)
note = note.sort_values(['CHARTDATE','CHARTTIME'], ascending = True)
note = note.sort_values('CHARTDATE', ascending = True)
note.dropna(subset=['HADM_ID'],inplace=True)

note = note.reset_index().drop('index',axis=1)
# print(len(note))
# print(f"number of unique patients with note : {note.SUBJECT_ID.nunique()}")
# print('\n\n',note.info())
# print('\n\n',note.isna().sum(),'\n')

In [ ]:
note['len_text_1'] = note['TEXT'].apply(lambda x: len(str(x).split(' ')))


In [ ]:
note['len_text_1'].describe()

## First day note

In [ ]:
df_note = note
df_note.head(2)

In [ ]:
df_note.dropna(subset=['CHARTDATE','CHARTTIME'],inplace=True)

In [ ]:
df_note['CHARTTIMESTAMP'] = pd.to_datetime(df_note['CHARTDATE'], errors='coerce').dt.strftime('%Y-%m-%d')
df_note['CHARTTIMESTAMP']  = df_note['CHARTTIMESTAMP'] + ' ' +pd.to_datetime(df_note['CHARTTIME'], errors='coerce').dt.strftime('%H:%M:%S')

In [ ]:
df_note['CHARTTIMESTAMP']

In [ ]:
# Define the 24 h cutoff
df_adm['ADMITTIME'] = pd.to_datetime(df_adm.ADMITTIME)
df_adm['CUTOFF_24H'] = df_adm.ADMITTIME + pd.Timedelta(hours=24)
# Join notes → admissions first
n = pd.merge(df_note, df_adm[['SUBJECT_ID','HADM_ID','ADMITTIME','CUTOFF_24H']], 
                 on=['SUBJECT_ID','HADM_ID'], how='inner')
# Filter to the window
notes_fd = n[ n.CHARTTIMESTAMP <= n.CUTOFF_24H ]
# Aggregate all text in that 24 h window per patient
df_note_fd = (notes_fd
           .groupby(['SUBJECT_ID','HADM_ID'])['TEXT']
           .apply(lambda texts: ' '.join(texts))
           .reset_index(name='FIRST_24H_TEXT'))

In [ ]:
notes_fd.head(2)

In [ ]:
print(len(notes_fd))

In [ ]:
df_note_fd.head(2)
df_note_fd['FIRST_24H_TEXT'].iloc[0]

In [ ]:
df_note_fd.to_csv('df_before_joining _text.csv')

In [ ]:
df_note_fd.head(2)

In [ ]:
df_note_fd = df_note.groupby(['SUBJECT_ID','HADM_ID', 'CHARTDATE'])['TEXT'].apply(lambda x: ' '.join(x)).reset_index()
df_note_fd.rename(columns={'TEXT':'FIRST_DAY_TEXT'}, inplace=True)
df_note_fd['len_text_2'] = df_note_fd['FIRST_DAY_TEXT'].apply(lambda x: len(str(x).split(' ')))
df_note_fd["CHARTDATE"] = pd.to_datetime(df_note_fd["CHARTDATE"], errors='coerce')

In [ ]:
df_note_fd['len_text_2'] = df_note_fd['FIRST_24H_TEXT'].apply(lambda x: len(str(x).split(' ')))

In [ ]:
print(len(df_note_fd))
df_note_fd.head(2)

In [ ]:
df_note_fd = df_note_fd.drop_duplicates(['SUBJECT_ID','HADM_ID'],keep='first')
print(len(df_note_fd))

In [ ]:
df_note_fd.len_text_2.describe()

In [ ]:
df_final = pd.merge(df_note_fd,df, how='inner', on=['SUBJECT_ID','HADM_ID'])
df_final['HOSPITAL_EXPIRE_FLAG_CAT']= df_final['HOSPITAL_EXPIRE_FLAG'].apply(lambda x: 'Yes' if x==1 else 'No')
df_final.head(2)

In [ ]:
print(len(df_final))

In [ ]:
df_final['len_text_2'].plot.hist(bins=300)
df_final['len_text_2'].describe()


In [ ]:
df_final['len_text_2'][(df_final.len_text_2 < 720) & (df_final.len_text_2 > 1)].plot.hist(bins=300)
df_final['len_text_2'][(df_final.len_text_2 < 720) & (df_final.len_text_2 > 1)].describe()

In [ ]:
df_final = df_final[(df_final.len_text_2 < 30000) & (df_final.len_text_2 > 70)]

In [ ]:
df_final.groupby('HOSPITAL_EXPIRE_FLAG').size()

In [ ]:
print(len(df_final[df_final['HOSPITAL_EXPIRE_FLAG']==0]))
df_ones = df_final[df_final['HOSPITAL_EXPIRE_FLAG'] == 1]

# Randomly sample rows with binary_column equal to 0 
df_zeros_sampled = df_final[df_final['HOSPITAL_EXPIRE_FLAG'] == 0].sample(850, random_state=42)

# Combine the two subsets
df_final_sample = pd.concat([df_ones, df_zeros_sampled]).reset_index(drop=True)
print(len(df_final_sample[df_final_sample['HOSPITAL_EXPIRE_FLAG']==0]))
print(df_final_sample.groupby('HOSPITAL_EXPIRE_FLAG').size())



In [ ]:
# save final cohort
# df_final_sample.to_excel('df_final_sample.xlsx',engine='openpyxl')

In [ ]:
import re

def remove_illegal_chars(val):
    if isinstance(val, str):
        return re.sub(r"[\x00-\x1F\x7F-\x9F]", "", val)
    return val

# Apply cleanup to all string values
df_final_sample = df_final_sample.applymap(remove_illegal_chars)

# Then write to Excel
df_final_sample.to_excel('df_final_sample.xlsx', engine='openpyxl')

## Analysis on the length of the notes

In [ ]:
len(df_final_sample['len_text_2'][(df_final_sample['HOSPITAL_EXPIRE_FLAG']==0) & (df_final_sample['len_text_2']>7000)])

In [ ]:
len(df_final_sample['len_text_2'][(df_final_sample['HOSPITAL_EXPIRE_FLAG']==1) & (df_final_sample['len_text_2']>7000)])

In [ ]:
df_final_sample['len_text_2'][df_final_sample['HOSPITAL_EXPIRE_FLAG']==0].describe()

In [ ]:
df_final_sample['len_text_2'].describe()

In [ ]:
# Assuming your DataFrame is df and token lengths are in the 'token_len' column
Q1 = df_final_sample['len_text_2'][df_final_sample['HOSPITAL_EXPIRE_FLAG']==0].quantile(0.25)
Q3 = df_final_sample['len_text_2'][df_final_sample['HOSPITAL_EXPIRE_FLAG']==0].quantile(0.75)
IQR = Q3 - Q1

upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
df_filtered_0 = ddf_filtered_0 = df_final_sample[
    (df_final_sample['HOSPITAL_EXPIRE_FLAG'] == 0) &
    (df_final_sample['len_text_2'] <= upper_bound)
]


print(f"Removed {len(df_final_sample['len_text_2'][df_final_sample['HOSPITAL_EXPIRE_FLAG']==0]) - len(df_filtered_0)} outlier notes.")

In [ ]:
# Assuming your DataFrame is df and token lengths are in the 'token_len' column
Q1 = df_final_sample['len_text_2'][df_final_sample['HOSPITAL_EXPIRE_FLAG']==1].quantile(0.25)
Q3 = df_final_sample['len_text_2'][df_final_sample['HOSPITAL_EXPIRE_FLAG']==1].quantile(0.75)
IQR = Q3 - Q1

upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
df_filtered_1 = ddf_filtered_0 = df_final_sample[
    (df_final_sample['HOSPITAL_EXPIRE_FLAG'] == 1) &
    (df_final_sample['len_text_2'] <= upper_bound)
]


print(f"Removed {len(df_final_sample['len_text_2'][df_final_sample['HOSPITAL_EXPIRE_FLAG']==1]) - len(df_filtered_1)} outlier notes.")

In [ ]:
df_filtered = pd.concat([df_filtered_0, df_filtered_1], ignore_index=True)
len(df_filtered)

In [ ]:
import re

def remove_illegal_chars(val):
    if isinstance(val, str):
        return re.sub(r"[\x00-\x1F\x7F-\x9F]", "", val)
    return val

# Apply cleanup to all string values
df_filtered = df_filtered.applymap(remove_illegal_chars)

# Then write to Excel
df_filtered.to_excel('df_filtered.xlsx', engine='openpyxl')

In [ ]:
print(df_filtered.groupby('HOSPITAL_EXPIRE_FLAG').size())
print(len(df_filtered))

In [ ]:
data = pd.read_excel('df_filtered.xlsx')
print(len(data))
print(data.groupby('HOSPITAL_EXPIRE_FLAG_CAT').size())

## Demographics

In [ ]:
## add race and ethnicity
path = "PATIENTS.csv.gz"
df_pt = pd.read_csv(path, compression='gzip', header=0)
df_pt.drop_duplicates(inplace=True)
df_pt.head(2)
data =pd.merge(df_filtered, df_pt[['SUBJECT_ID', 'GENDER', 'DOB', 'DOD']], how='inner', on='SUBJECT_ID')
data.drop_duplicates('SUBJECT_ID', inplace=True, keep='first')
# data.dropna(subset=['ETHNICITY'],inplace=True)
data['ETHNICITY'][data.ETHNICITY.isna()] = 'UNKNOWN (DEFAULT)'
data['MARITAL_STATUS'][data.MARITAL_STATUS.isna()] = 'UNKNOWN (DEFAULT)'
print(len(data))
data.head(2)

In [ ]:
data['ETHNICITY'][data.ETHNICITY.str.contains('WHITE')]='WHITE'
data['ETHNICITY'][data.ETHNICITY.str.contains('ASIAN')] = 'ASIAN'
data['ETHNICITY'][data.ETHNICITY.str.contains('HISPANIC')] = 'HISPANIC'
data['ETHNICITY'][data.ETHNICITY.str.contains('CARIBBEAN ISLAND')] = 'HISPANIC'
data['ETHNICITY'][data.ETHNICITY.str.contains('SOUTH AMERICAN')] = 'HISPANIC'
data['ETHNICITY'][data.ETHNICITY.str.contains('PORTUGUESE')] = 'OTHER'
data['ETHNICITY'][data.ETHNICITY.str.contains('AMERICAN INDIAN')] = 'AMERICAN INDIAN'
data['ETHNICITY'][data.ETHNICITY.str.contains('BLACK')] = 'BLACK'
data['ETHNICITY'][data.ETHNICITY.isin(['UNKNOWN/NOT SPECIFIED','UNABLE TO OBTAIN','PATIENT DECLINED TO ANSWER'])]='UNABLE TO OBTAIN'

g = data.groupby(['GENDER','HOSPITAL_EXPIRE_FLAG_CAT']).size()
print(round(g/g.sum(),2)*100)
g = data.groupby(['ETHNICITY','HOSPITAL_EXPIRE_FLAG_CAT']).size()
print(round(g/g.sum(),4)*100)
g = data.dropna(subset=['MARITAL_STATUS']).groupby(['MARITAL_STATUS','HOSPITAL_EXPIRE_FLAG_CAT']).size()
print(round(g/g.sum(),4)*100)

In [ ]:
# GENDER distribution
g_gender = data.groupby(['GENDER', 'HOSPITAL_EXPIRE_FLAG_CAT']).size()
g_gender_pct = round(g_gender / g_gender.sum(), 2) * 100
gender_df = g_gender_pct.reset_index(name='Percentage')
gender_df['Category'] = 'Gender'

# ETHNICITY distribution
g_ethnicity = data.groupby(['ETHNICITY', 'HOSPITAL_EXPIRE_FLAG_CAT']).size()
g_ethnicity_pct = round(g_ethnicity / g_ethnicity.sum(), 4) * 100
ethnicity_df = g_ethnicity_pct.reset_index(name='Percentage')
ethnicity_df['Category'] = 'Ethnicity'

# MARITAL_STATUS distribution (drop NaNs first)
g_marital = data.dropna(subset=['MARITAL_STATUS']).groupby(['MARITAL_STATUS', 'HOSPITAL_EXPIRE_FLAG_CAT']).size()
g_marital_pct = round(g_marital / g_marital.sum(), 4) * 100
marital_df = g_marital_pct.reset_index(name='Percentage')
marital_df['Category'] = 'Marital Status'

# Harmonize column names
gender_df.rename(columns={'GENDER': 'Group'}, inplace=True)
ethnicity_df.rename(columns={'ETHNICITY': 'Group'}, inplace=True)
marital_df.rename(columns={'MARITAL_STATUS': 'Group'}, inplace=True)

# Concatenate all into one DataFrame
final_df = pd.concat([gender_df, ethnicity_df, marital_df], ignore_index=True)

# Optional: Sort for readability
final_df = final_df[['Category', 'Group', 'HOSPITAL_EXPIRE_FLAG_CAT', 'Percentage']]
final_df.sort_values(by=['Category', 'Group'], inplace=True)
display(final_df)
final_df.to_csv('demograghics.csv')

In [ ]:
# Pivot the data
pivot_df = final_df.pivot_table(
    index='Group',
    columns=['Category', 'HOSPITAL_EXPIRE_FLAG_CAT'],
    values='Percentage'
)

# Optional: sort the index and columns for cleaner presentation
pivot_df = pivot_df.sort_index().sort_index(axis=1)

# Display the pivoted DataFrame
display(pivot_df)

## Please ensure prompting_GPT.py has been executed before running this section.

In [ ]:
## Functions

# Convert string to list
def str_to_list(x):

    return ast.literal_eval(x)

def most_comm(x):
    return x.most_common(1)[0]

def least_common(x):
#     print(x.most_common(1)[0])
    c = x.most_common(1)[0]
#     print(c[1])
    if c[0].lower() == 'yes':
        return ('No',30-c[1])
    else:
        return ('Yes',30-c[1])
    

In [ ]:
data = pd.read_excel('df_filtered_gpt_temp_1.xlsx').drop('Unnamed: 0', axis=1)
data.head(2)

In [ ]:
data.info()

In [ ]:
import math

# Convert string to list
def str_to_list(x):

    return ast.literal_eval(x)


def most_comm(lst):
    data = Counter(lst)
    return data.most_common(1)[0][0]



def least_common(x):
#     print(x.most_common(1)[0])
    c = x.most_common(1)[0]
#     print(c[1])
    if c[0].lower() == 'yes':
        return ('No',30-c[1])
    else:
        return ('Yes',30-c[1])
    

def normalized_yes_probability(lst):
    
    
    top_prediction_and_normalized_prob = []
    
    
    y_n_o_probs_all = []
    
    for entries in lst:
        y_n_o_probs = []
        # Convert logprobs to standard probabilities
        probs = [(text, math.exp(logprob)) for text, logprob in entries]
        
        # Total probability for normalization
        total_prob = sum(prob for _, prob in probs)
        
        # Sum of probabilities for entries containing "yes" (case-insensitive)
        yes_prob = sum(prob for text, prob in probs if 'yes' in text.lower())
       
        
        # Sum of probabilities for entries containing "no" (case-insensitive)
        no_prob = sum(prob for text, prob in probs if 'no' in text.lower())
       
        
        # Normalized probability
        y_n_o_probs.append(no_prob / total_prob if total_prob > 0 else 0.0)
        y_n_o_probs.append(yes_prob / total_prob if total_prob > 0 else 0.0)
        o_prob = 1- y_n_o_probs[0] - y_n_o_probs[1]
        y_n_o_probs.append (o_prob)
#         print('yes no other prob: ', y_n_o_probs)
        
        y_n_o_probs_all.append(y_n_o_probs)
       
        #print('all:  ', y_n_o_probs_all)
#         input()


        
        top_pred = max(probs, key=lambda x: x[1])
        if False:
            print('Top 10 logprobs:\n', entries, '\n')
            print('Top 10 logprobs => probs:\n', probs, '\n')
            print('total of all 10 probs for normalization:\n', total_prob, '\n')
            print('yes prob (sum of all containing yes):\n', yes_prob, '\n')
            print('text + prob (max among those containing yes):\n', max([x for x in probs if 'yes' in x[0].lower()], key=lambda x: x[1]), '\n')
            print('no prob (sum of all containing no):\n', no_prob, '\n')
            print('text + prob (max among those containing no):\n', max([x for x in probs if 'no' in x[0].lower()], key=lambda x: x[1]), '\n')
            print('Response with Max Probability:\n\n', top_pred, '\n')
        # normalize prob
        top_pred = (top_pred[0], (top_pred[1] / (total_prob if total_prob > 0 else 1)))
        top_prediction_and_normalized_prob.append(top_pred)

        
        
    return y_n_o_probs_all
    # return y_n_o_probs_all, top_prediction_and_normalized_prob


def sample_final_prediction(lst):
    l=[]
    for item in lst:
        l.append(item.index(max(item)))
#     print(l)
    return l   

def true_class_prob (x):
    return [sublist[x[1]] for sublist in x[0]]
    

In [ ]:
print(data.groupby('HOSPITAL_EXPIRE_FLAG').size())
print(len(data['prediction_logprob'].apply(str_to_list).iloc[0]), len(data['prediction_logprob'].apply(str_to_list).iloc[0][0]))

data['prediction_logprob'] = data['prediction_logprob'].apply(str_to_list)
data ['y_n_o_probs_all'] = data['prediction_logprob'].apply(normalized_yes_probability)
# sample_final_prediction(data ['y_n_o_probs_all'][0])
data ['sample_final_prediction'] = data ['y_n_o_probs_all'].apply(sample_final_prediction)
data ['final_prediction'] = data ['sample_final_prediction'].apply(most_comm)
data['HOSPITAL_EXPIRE_FLAG_CAT']= data['HOSPITAL_EXPIRE_FLAG'].apply(lambda x: 'Yes' if x==1 else 'No')
data['true_class_prob'] = data[['y_n_o_probs_all','HOSPITAL_EXPIRE_FLAG']].apply(true_class_prob,axis=1)
data['nonconformity_scores'] = data['true_class_prob'].apply(lambda lst: [1-p for p in lst])
all_nonconformity_scores = sum(data['nonconformity_scores'], [])

In [ ]:
# Split the data into a calibration set and a test set (e.g., 50% each)
cal = data.sample(frac=0.5, random_state=42)
test = data.drop(cal.index).reset_index(drop=True)

## save caliberation and test dataset
# cal.to_csv('calibration_gpt_temp_1_v2.csv')
# test.to_excel('test_gpt_temp_1_v2.xlsx')

In [ ]:
# Calculate evaluation metrics
accuracy = round(accuracy_score(data['HOSPITAL_EXPIRE_FLAG'], data['final_prediction'])*100,2)
precision = round(precision_score(data['HOSPITAL_EXPIRE_FLAG'], data['final_prediction'])*100,2)
recall = round(recall_score(data['HOSPITAL_EXPIRE_FLAG'], data['final_prediction'])*100,2)
f1 = round(f1_score(data['HOSPITAL_EXPIRE_FLAG'], data['final_prediction'])*100,2)


# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
print(data.groupby(['HOSPITAL_EXPIRE_FLAG','final_prediction']).size())

## Error Analysis

## Accurate

In [ ]:
accurate_rows = data[data['HOSPITAL_EXPIRE_FLAG'] == data['final_prediction']]
accurate_rows.groupby(['HOSPITAL_EXPIRE_FLAG','final_prediction']).size()

In [ ]:
print(len(accurate_rows))

## Non Accurate


In [ ]:
non_accurate_rows = data[data['HOSPITAL_EXPIRE_FLAG'] != data['final_prediction']]
non_accurate_rows.groupby(['HOSPITAL_EXPIRE_FLAG','final_prediction']).size()

In [ ]:
import matplotlib.pyplot as plt
accurate_rows['len_text_2'].hist(bins= 20)
plt.xlabel('number of tokens per note')
plt.ylabel('frequency')
accurate_rows['len_text_2'].describe().round()

In [ ]:
# Plot histogram for the first set
plt.hist(accurate_rows['len_text_2'], bins=50, color='blue', alpha=0.5, label='Accuate prediction')

# Plot a second histogram (for example, another column or subset) with a different color
# Here, we assume you're overlaying another histogram from the same DataFrame.
# Replace 'other_column' with your actual column or subset if needed.
plt.hist(non_accurate_rows['len_text_2'], bins=50, color='red', alpha=0.5, label='Non-Accuate prediction')

plt.xlabel('Number of tokens per note')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
# Plot histogram for the first set
plt.figure()
plt.hist(accurate_rows['len_text_2'][accurate_rows['HOSPITAL_EXPIRE_FLAG']==1], bins=50, color='blue', alpha=0.5, label='Accuate prediction')
plt.xlabel('Number of tokens per note')
plt.ylabel('Frequency')
plt.title('Accurate in hospital mortality (flag=1)')
plt.show()

# Second histogram in a separate window
plt.figure()
plt.hist(non_accurate_rows['len_text_2'][non_accurate_rows['HOSPITAL_EXPIRE_FLAG']==1], bins=50, color='red', alpha=0.5, label='Non-Accuate prediction')

plt.xlabel('Number of tokens per note')
plt.ylabel('Frequency')
plt.title('Non-Accurate in hospital mortality (flag=1)')
plt.show()
plt.show()


print(accurate_rows['len_text_2'][accurate_rows['HOSPITAL_EXPIRE_FLAG']==1].describe().round())
print(non_accurate_rows['len_text_2'][non_accurate_rows['HOSPITAL_EXPIRE_FLAG']==1].describe().round())

In [ ]:
# Plot histogram for the first set
plt.figure()
plt.hist(accurate_rows['len_text_2'][accurate_rows['HOSPITAL_EXPIRE_FLAG']==0], bins=50, color='blue', alpha=0.5, label='Accuate prediction')
plt.xlabel('Number of tokens per note')
plt.ylabel('Frequency')
plt.title('Accurate in hospital mortality (flag=0)')
plt.show()

# Second histogram in a separate window
plt.figure()
plt.hist(non_accurate_rows['len_text_2'][non_accurate_rows['HOSPITAL_EXPIRE_FLAG']==0], bins=50, color='red', alpha=0.5, label='Non-Accuate prediction')

plt.xlabel('Number of tokens per note')
plt.ylabel('Frequency')
plt.title('Non-Accurate in hospital mortality (flag=0)')
plt.show()
plt.show()

print('accurate')
print(accurate_rows['len_text_2'][accurate_rows['HOSPITAL_EXPIRE_FLAG']==0].describe().round())

print('non-accurate')
print(non_accurate_rows['len_text_2'][non_accurate_rows['HOSPITAL_EXPIRE_FLAG']==0].describe().round())

In [ ]:
mortality_flag_1 = data[data['HOSPITAL_EXPIRE_FLAG'] == 1]
mortality_flag_1.groupby(['HOSPITAL_EXPIRE_FLAG','final_prediction']).size()

In [ ]:
mortality = mortality_flag_1[['len_text_2','HOSPITAL_EXPIRE_FLAG','final_prediction']]
mortality.head(2)

## t-test

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
correct = mortality[mortality['final_prediction'] == 1]['len_text_2']
incorrect = mortality[mortality['final_prediction'] == 0]['len_text_2']

# Perform an independent samples t-test
t_stat, p_value = ttest_ind(correct, incorrect)

print("T-test results:")
print("t-statistic:", t_stat)
print("p-value:", p_value)

In [ ]:
mortality_flag_0 = data[data['HOSPITAL_EXPIRE_FLAG'] == 0]
mortality_flag_0.groupby(['HOSPITAL_EXPIRE_FLAG','final_prediction']).size()

In [ ]:
non_mortality = mortality_flag_0[['len_text_2','HOSPITAL_EXPIRE_FLAG','final_prediction']]
correct = non_mortality[non_mortality['final_prediction'] == 0]['len_text_2']
incorrect = non_mortality[non_mortality['final_prediction'] == 1]['len_text_2']

# Perform an independent samples t-test
t_stat, p_value = ttest_ind(correct, incorrect)

print("T-test results:")
print("t-statistic:", t_stat)
print("p-value:", p_value)

In [ ]:
mortality_flag_1 = data[data['HOSPITAL_EXPIRE_FLAG'] == 1]
mortality_flag_1.groupby(['HOSPITAL_EXPIRE_FLAG','final_prediction']).size()

In [ ]:
mortality_flag_1_100 = mortality_flag_1[mortality_flag_1['len_text_2']<=100]
mortality_flag_1_100.groupby(['HOSPITAL_EXPIRE_FLAG','final_prediction']).size()

In [ ]:
mortality_flag_1_1 = mortality_flag_1[mortality_flag_1['final_prediction']==1]
mortality_flag_1_1.groupby(['HOSPITAL_EXPIRE_FLAG','final_prediction']).size()

In [ ]:
mortality_flag_1_0 = mortality_flag_1[mortality_flag_1['final_prediction']==0]
mortality_flag_1_0.groupby(['HOSPITAL_EXPIRE_FLAG','final_prediction']).size()

## NLP

In [ ]:
stops = set(stopwords.words('english'))
print(type(stops))
# stop = text.ENGLISH_STOP_WORDS.union(["book"])
stops.update('pt clinic work without with vs within'.split())
# Initialize stemmer
stemmer = PorterStemmer()


## Functions
def stemmed_words(doc):
    return ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(doc)])

def remove_punctuation(doc):
    exclude = set(string.punctuation)
    return ' '.join([word for word in nltk.word_tokenize(doc) if word not in exclude])
    


In [ ]:
all_text = " ".join(mortality_flag_1_1['FIRST_DAY_TEXT'].astype(str).tolist())

# Initialize stemmer
stemmer = PorterStemmer()

# Stemming function
def stemmed_words(doc):
    return ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(doc)])

# Apply stemming to the documents
stemmed_documents = [stemmed_words(doc) for doc in mortality_flag_1_1['FIRST_DAY_TEXT']]


vect = TfidfVectorizer(ngram_range=(2,2),lowercase=True,stop_words=set(stops))
matrix = vect.fit_transform(stemmed_documents)
feature_names = vect.get_feature_names()


# Create a word cloud from frequencies
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(vect.vocabulary_)

# Plot the word cloud
plt.figure(figsize=(15, 7.5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Word Cloud: Most Frequent Words Larger, Less Frequent Words Smaller")
plt.show()

In [ ]:
# Initialize stemmer
stemmer = PorterStemmer()

# Stemming function
def stemmed_words(doc):
    return ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(doc)])

# Apply stemming to the documents
stemmed_documents = [stemmed_words(doc) for doc in mortality_flag_1_0['FIRST_DAY_TEXT']]



vect = TfidfVectorizer(ngram_range=(2,2),lowercase=True,stop_words=set(stops))
matrix = vect.fit_transform(stemmed_documents)
feature_names = vect.get_feature_names()


# Create a word cloud from frequencies
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(vect.vocabulary_)

# Plot the word cloud
plt.figure(figsize=(15, 7.5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Word Cloud: Most Frequent Words Larger, Less Frequent Words Smaller")
plt.show()

In [ ]:
def plot_cloud(doc):

    # Remove punctuation
    doc = remove_punctuation(doc)
    
    # Apply stemming to the documents
    doc = stemmed_words(doc)


    vect = TfidfVectorizer(ngram_range=(1,2),lowercase=True,stop_words=set(stops))
    matrix = vect.fit_transform(doc)
    feature_names = vect.get_feature_names()


    # Create a word cloud from frequencies
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(vect.vocabulary_)

    # Plot the word cloud
    plt.figure(figsize=(15, 7.5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title("Word Cloud: Most Frequent Words Larger, Less Frequent Words Smaller")
    plt.show()